## SageMaker Training Job 

### Please go through this notebook only if you have finished Part 1 to Part 4 of the tutorial.

---
#### Step 1: Import packages, get IAM role, get the region and set the S3 bucket.

In [ ]:
%%sh 
pwd
cd local_test/test_dir/input/data/training/
ls -ltr
aws s3 cp data_set link to s3 bucket containing dataset

---
#### Step 2: Create the algorithm image and push to Amazon ECR.

In [ ]:
#Do not execute this
%%sh

chmod +x create_container.sh 

./create_container.sh keras-sagemaker

---
#### Step 3: Define variables with data location and output location in S3 bucket.

In [1]:
schema = 's3:/' 
bucket = 'eu.com.syngenta-datascience-model-training'

user = 'username' 
experiment = 'keras-sagemaker'


data_location = f'{schema}/{bucket}/{user}/{ experiment}/train/data'
print("data location - " + data_location)

output_location = f'{schema}/{bucket}/{user}/{ experiment}/output'
print("output location - " + output_location)

data location - s3://link to dataset
output location - s3://link to output


---
#### Step 4: Create a SageMaker session.

In [2]:
import sagemaker as sage

sagemaker_session = sage.Session()

# this line of code requires iam:GetRole permissions
#role = sage.get_execution_role()

---
#### Step 5: Define variables for account, region and algorithm image.

In [4]:
account = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account'] # aws account 
region = sagemaker_session.boto_session.region_name # aws server region
tag='cpu'
image = '{}.dkr.ecr.{}.amazonaws.com/datascience-model-training:{}'.format(account, region, tag) # algorithm image path in ECR
#keras-sagemaker-train
print(account)
print(region)
print(image)

170605107178
eu-central-1
170605107178.dkr.ecr.eu-central-1.amazonaws.com/datascience-model-training:cpu


---
#### Step 6: Define hyperparameters to be passed to your algorithm. 
In this project we are reading two hyperparameters for training. Use of hyperparameters in optional.

In [5]:
hyperparameters = {"batch_size":128, "epochs":30}

---
#### Step 7: Create the training job using SageMaker Estimator.

In [6]:
role = 'arn:aws:iam::170605107178:role/SYN-Datascience-SageMaker-Role'

subnets_config = ['subnet-0bdd33f41f946b22a', 'subnet-0c7c8959343746db7']

security_groups_config = [ "sg-1ad4ea70",
              "sg-99d781f1",
              "sg-dfa1f7b7"]

# the instance type to be used for training. using 'local' will not trigger a job on SageMaker
instance_count = 1
instance_type = 'ml.p3.2xlarge'

#https://aws.amazon.com/sagemaker/pricing/
#https://aws.amazon.com/sagemaker/pricing/instance-types/

#ml.p3.2xlarge Accelerated Computing – Current Generation 	8(CPU cores)	1xV100(GPU)	61GiB(CPU mem)	16GiB(Gpu mem)
# ml.c5.2xlarge 	  Compute Instances - Current Generation	 $0.543 16GiB RAM 8 cores
# ml.m4.xlarge	   Compute Instances - Standard Generation	 $0.336 16GiB Ram 4 cores


classifier = sage.estimator.Estimator(image_name=image, 
                                      role=role,
                                      train_instance_count=instance_count, 
                                      train_instance_type= instance_type,
                                      hyperparameters=hyperparameters,
                                      output_path=output_location,
                                      subnets=subnets_config, 
                                      security_group_ids=security_groups_config,
                                      sagemaker_session=sagemaker_session)

---
#### Step 8: Run the training job by passing the data location.

In [7]:
from datetime import datetime

job_prefix_name = 'datascience-model-training'
training_job_name = job_prefix_name + '-'  + datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

print(training_job_name)

datascience-model-training-2020-04-17-07-38-43


In [8]:
classifier.fit(inputs=data_location, 
              logs=True, 
              job_name=training_job_name)

2020-04-17 07:38:47 Starting - Starting the training job...
2020-04-17 07:38:49 Starting - Launching requested ML instances......
2020-04-17 07:40:13 Starting - Preparing the instances for training......
2020-04-17 07:41:21 Downloading - Downloading input data
2020-04-17 07:41:21 Training - Downloading the training image......
2020-04-17 07:42:16 Training - Training image download completed. Training in progress..2020-04-17 07:42:19.578385: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-04-17 07:42:19.637425: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300065000 Hz
2020-04-17 07:42:19.637901: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56247946f1e0 executing computations on platform Host. Devices:
2020-04-17 07:42:19.637933: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2020-04

8000/8000 [==============================] - 0s 61us/step - loss: 0.3129 - accuracy: 0.9044 - val_loss: 0.2641 - val_accuracy: 0.9270
Epoch 13/30

8000/8000 [==============================] - 0s 61us/step - loss: 0.3033 - accuracy: 0.9084 - val_loss: 0.2330 - val_accuracy: 0.9360
Epoch 14/30

8000/8000 [==============================] - 0s 61us/step - loss: 0.2869 - accuracy: 0.9128 - val_loss: 0.2222 - val_accuracy: 0.9355
Epoch 15/30

8000/8000 [==============================] - 0s 61us/step - loss: 0.2775 - accuracy: 0.9154 - val_loss: 0.2091 - val_accuracy: 0.9405
Epoch 16/30

8000/8000 [==============================] - 0s 61us/step - loss: 0.2619 - accuracy: 0.9180 - val_loss: 0.2046 - val_accuracy: 0.9395
Epoch 17/30

8000/8000 [==============================] - 0s 60us/step - loss: 0.2518 - accuracy: 0.9255 - val_loss: 0.1989 - val_accuracy: 0.9445
Epoch 18/30

8000/8000 [==============================] - 0s 61us/step - loss: 0.2414 - accuracy: 0.9262 - val_loss: 0.2306 - val_a

8000/8000 [==============================] - 0s 61us/step - loss: 0.2046 - accuracy: 0.9386 - val_loss: 0.1681 - val_accuracy: 0.9525
Epoch 23/30

8000/8000 [==============================] - 0s 60us/step - loss: 0.1929 - accuracy: 0.9402 - val_loss: 0.1809 - val_accuracy: 0.9495
Epoch 24/30

8000/8000 [==============================] - 0s 62us/step - loss: 0.1870 - accuracy: 0.9424 - val_loss: 0.1683 - val_accuracy: 0.9505
Epoch 25/30

8000/8000 [==============================] - 0s 61us/step - loss: 0.1809 - accuracy: 0.9424 - val_loss: 0.1718 - val_accuracy: 0.9505
Epoch 26/30

8000/8000 [==============================] - 0s 60us/step - loss: 0.1729 - accuracy: 0.9479 - val_loss: 0.1851 - val_accuracy: 0.9450
Epoch 27/30

8000/8000 [==============================] - 1s 63us/step - loss: 0.1646 - accuracy: 0.9476 - val_loss: 0.1552 - val_accuracy: 0.9535
Epoch 28/30

8000/8000 [==============================] - 0s 61us/step - loss: 0.1592 - accuracy: 0.9520 - val_loss: 0.1617 - val_a


2020-04-17 07:42:53 Uploading - Uploading generated training model
2020-04-17 07:42:53 Completed - Training job completed
Training seconds: 108
Billable seconds: 108


## Congratulations! We had a successful training job run in Amazon SageMaker.
#### Please return to the tutorial for Part 6 where we will be running a training job in a GPU.